<a href="https://colab.research.google.com/github/vit050587/Natural_Language_Processing/blob/master/KVA_HW_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ДЗ 8. Рекуррентные нейронные сети RNN LSTM GRU
Задание
Данные берем отызывы за лето.

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи.

* построить свёрточные архитектуры
* построить различные архитектуры с RNN
* построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymorphy2
!pip install stop_words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 65.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=70ba6b1d040aac15cd65dedd658a42ca788f87c3d79579bd079fb96cacf2fe14
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32894 sha256=af1717a87c60d050b9a8881b3ab4b35c0811f38fb920424a20036e925d8aed33
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [3]:
# Импорт библиотек

import pandas as pd
import numpy as np
import re
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from gensim.models import Word2Vec, FastText

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, GlobalMaxPool2D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

from sklearn.metrics import classification_report

In [4]:
!pip install --upgrade xlrd

In [5]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/lesson07/lection7/отзывы за лето.xls')

In [6]:
df

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [7]:
df = df.dropna()

In [8]:
df.shape

(20656, 3)

In [9]:
df.Rating.value_counts()

5    14584
1     2276
4     2137
3      911
2      748
Name: Rating, dtype: int64

In [10]:
# делаем колонку Content текстовой - так как ругается что - 'int' object has no attribute 'translate'

df.Content = df.Content.apply(lambda text: str(text))

<ipython-input-10-26a918284a53>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Content = df.Content.apply(lambda text: str(text))


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Content, df.Rating, test_size=0.3, random_state=42, stratify=df.Rating)
y_train.value_counts()

5    10209
1     1593
4     1496
3      638
2      523
Name: Rating, dtype: int64

In [12]:
y_test.value_counts()

5    4375
1     683
4     641
3     273
2     225
Name: Rating, dtype: int64

In [13]:
corpus_train = X_train
corpus_test = X_test

In [14]:
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(corpus_train)

sequences_train = tokenizer.texts_to_sequences(corpus_train)
sequences_test = tokenizer.texts_to_sequences(corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

#Построение сетей

In [15]:
num_classes = 6

epochs = 20
batch_size = 512

comparing_list = []

In [16]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

#CNN+pooling

In [17]:
net_name = 'CNN+pooling'

model = Sequential()
model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
# model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping],
                    )

Epoch 1/20
26/26 [==============================] - 39s 1s/step - loss: 1.7132 - accuracy: 0.0423 - val_loss: 1.6260 - val_accuracy: 0.0443
Epoch 2/20
26/26 [==============================] - 24s 892ms/step - loss: 1.5525 - accuracy: 0.2863 - val_loss: 1.4633 - val_accuracy: 0.7026
Epoch 3/20
26/26 [==============================] - 21s 829ms/step - loss: 1.3288 - accuracy: 0.7064 - val_loss: 1.2101 - val_accuracy: 0.7033
Epoch 4/20
26/26 [==============================] - 22s 817ms/step - loss: 1.0054 - accuracy: 0.7470 - val_loss: 0.9267 - val_accuracy: 0.7580
Epoch 5/20
26/26 [==============================] - 20s 784ms/step - loss: 0.8089 - accuracy: 0.7820 - val_loss: 0.8470 - val_accuracy: 0.7635
Epoch 6/20
26/26 [==============================] - 24s 920ms/step - loss: 0.6994 - accuracy: 0.7905 - val_loss: 0.8165 - val_accuracy: 0.7642
Epoch 7/20
26/26 [==============================] - 20s 774ms/step - loss: 0.6096 - accuracy: 0.8058 - val_loss: 0.7833 - val_accuracy: 0.7600
Ep

In [18]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 2s 12ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.47      0.81      0.59       683
           2       0.14      0.40      0.21       225
           3       0.07      0.70      0.12       273
           4       0.18      0.69      0.28       641
           5       0.83      0.96      0.89      4375

   micro avg       0.45      0.88      0.59      6197
   macro avg       0.28      0.59      0.35      6197
weighted avg       0.66      0.88      0.73      6197
 samples avg       0.58      0.88      0.66      6197



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#RNN

In [19]:
net_name = 'RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 18s 625ms/step - loss: 1.5597 - accuracy: 0.5777 - val_loss: 1.1336 - val_accuracy: 0.7026
Epoch 2/20
26/26 [==============================] - 17s 644ms/step - loss: 0.9369 - accuracy: 0.7256 - val_loss: 0.7656 - val_accuracy: 0.7580
Epoch 3/20
26/26 [==============================] - 18s 678ms/step - loss: 0.6719 - accuracy: 0.7815 - val_loss: 0.7025 - val_accuracy: 0.7676
Epoch 4/20
26/26 [==============================] - 18s 685ms/step - loss: 0.5156 - accuracy: 0.8310 - val_loss: 0.7093 - val_accuracy: 0.7559


In [20]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 3s 16ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.40      0.83      0.54       683
           2       0.14      0.33      0.19       225
           3       0.17      0.37      0.24       273
           4       0.10      0.86      0.18       641
           5       0.84      0.96      0.90      4375

   micro avg       0.42      0.89      0.57      6197
   macro avg       0.28      0.56      0.34      6197
weighted avg       0.66      0.89      0.73      6197
 samples avg       0.43      0.89      0.58      6197



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#CNN + RNN

In [21]:
net_name = 'CNN+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3, activation='relu'))
# model.add(GlobalMaxPool1D(keepdims=True))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

# print(model.summary())
# plot_model(model, show_shapes=True, show_layer_names=True)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 32s 1s/step - loss: 1.2723 - accuracy: 0.6160 - val_loss: 0.8739 - val_accuracy: 0.7068
Epoch 2/20
26/26 [==============================] - 28s 1s/step - loss: 0.8940 - accuracy: 0.7287 - val_loss: 0.7635 - val_accuracy: 0.7414
Epoch 3/20
26/26 [==============================] - 29s 1s/step - loss: 0.7457 - accuracy: 0.7718 - val_loss: 0.7178 - val_accuracy: 0.7559
Epoch 4/20
26/26 [==============================] - 29s 1s/step - loss: 0.6084 - accuracy: 0.8000 - val_loss: 0.6889 - val_accuracy: 0.7586
Epoch 5/20
26/26 [==============================] - 30s 1s/step - loss: 0.4955 - accuracy: 0.8363 - val_loss: 0.7195 - val_accuracy: 0.7476


In [22]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 4s 21ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.37      0.83      0.51       683
           2       0.14      0.72      0.23       225
           3       0.15      0.70      0.25       273
           4       0.10      0.99      0.19       641
           5       0.82      0.97      0.89      4375

   micro avg       0.38      0.93      0.54      6197
   macro avg       0.26      0.70      0.34      6197
weighted avg       0.64      0.93      0.72      6197
 samples avg       0.42      0.93      0.57      6197



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#CNN+pooling+RNN

In [23]:
net_name = 'CNN+pooling+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D(keepdims=True))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

# print(model.summary())
# plot_model(model, show_shapes=True, show_layer_names=True)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 24s 863ms/step - loss: 1.3148 - accuracy: 0.6709 - val_loss: 0.9453 - val_accuracy: 0.7026
Epoch 2/20
26/26 [==============================] - 20s 770ms/step - loss: 0.8534 - accuracy: 0.7296 - val_loss: 0.7123 - val_accuracy: 0.7607
Epoch 3/20
26/26 [==============================] - 22s 852ms/step - loss: 0.6774 - accuracy: 0.7762 - val_loss: 0.6632 - val_accuracy: 0.7725
Epoch 4/20
26/26 [==============================] - 20s 771ms/step - loss: 0.5725 - accuracy: 0.8052 - val_loss: 0.6449 - val_accuracy: 0.7739
Epoch 5/20
26/26 [==============================] - 21s 811ms/step - loss: 0.4858 - accuracy: 0.8330 - val_loss: 0.6690 - val_accuracy: 0.7704


In [24]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 3s 15ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.42      0.86      0.56       683
           2       0.14      0.80      0.23       225
           3       0.14      0.81      0.24       273
           4       0.11      0.95      0.19       641
           5       0.83      0.97      0.89      4375

   micro avg       0.39      0.94      0.55      6197
   macro avg       0.27      0.73      0.35      6197
weighted avg       0.65      0.94      0.73      6197
 samples avg       0.42      0.94      0.58      6197



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#RNN + CNN + CNN

In [25]:
net_name = 'RNN+CNN+CNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
model.add(SimpleRNN(64, recurrent_dropout=0.2, return_sequences="True"))
model.add(Conv1D(64, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear"))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="sigmoid"))


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 31s 1s/step - loss: 0.3884 - accuracy: 0.6811 - val_loss: 0.2355 - val_accuracy: 0.7158
Epoch 2/20
26/26 [==============================] - 28s 1s/step - loss: 0.2283 - accuracy: 0.7310 - val_loss: 0.2107 - val_accuracy: 0.7469
Epoch 3/20
26/26 [==============================] - 29s 1s/step - loss: 0.1962 - accuracy: 0.7642 - val_loss: 0.1991 - val_accuracy: 0.7545
Epoch 4/20
26/26 [==============================] - 29s 1s/step - loss: 0.1694 - accuracy: 0.7919 - val_loss: 0.1987 - val_accuracy: 0.7580
Epoch 5/20
26/26 [==============================] - 29s 1s/step - loss: 0.1495 - accuracy: 0.8183 - val_loss: 0.2050 - val_accuracy: 0.7476


In [26]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 4s 21ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.52      0.20      0.29       683
           2       0.00      0.00      0.00       225
           3       0.00      0.00      0.00       273
           4       0.14      0.00      0.01       641
           5       0.89      0.93      0.91      4375

   micro avg       0.86      0.68      0.76      6197
   macro avg       0.26      0.19      0.20      6197
weighted avg       0.70      0.68      0.68      6197
 samples avg       0.68      0.68      0.68      6197



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#RNN CNN pooling RNN

In [27]:
net_name = 'RNN+CNN+pooling+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D(keepdims=True)) # keepdims Сохраняет размерность для SimpleRNN
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 48s 2s/step - loss: 1.2263 - accuracy: 0.6938 - val_loss: 0.7824 - val_accuracy: 0.7441
Epoch 2/20
26/26 [==============================] - 44s 2s/step - loss: 0.7553 - accuracy: 0.7531 - val_loss: 0.7001 - val_accuracy: 0.7635
Epoch 3/20
26/26 [==============================] - 43s 2s/step - loss: 0.6438 - accuracy: 0.7786 - val_loss: 0.7036 - val_accuracy: 0.7635


In [28]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 7s 34ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.35      0.94      0.51       683
           2       0.15      0.48      0.23       225
           3       0.15      0.75      0.25       273
           4       0.10      0.91      0.18       641
           5       0.82      0.97      0.89      4375

   micro avg       0.39      0.93      0.55      6197
   macro avg       0.26      0.67      0.34      6197
weighted avg       0.64      0.93      0.72      6197
 samples avg       0.42      0.93      0.57      6197



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#(CNN RNN)x3 + polling

In [29]:
net_name = '(CNN+RNN)x3+polling'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3, activation='relu'))
model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(64, 3, activation='relu'))
model.add(SimpleRNN(64, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(32, 2, activation='relu'))
model.add(SimpleRNN(32, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 64s 2s/step - loss: 1.3230 - accuracy: 0.6006 - val_loss: 0.9085 - val_accuracy: 0.7344
Epoch 2/20
26/26 [==============================] - 58s 2s/step - loss: 0.9503 - accuracy: 0.7084 - val_loss: 0.7751 - val_accuracy: 0.7483
Epoch 3/20
26/26 [==============================] - 58s 2s/step - loss: 0.8240 - accuracy: 0.7410 - val_loss: 0.7452 - val_accuracy: 0.7600
Epoch 4/20
26/26 [==============================] - 57s 2s/step - loss: 0.7500 - accuracy: 0.7566 - val_loss: 0.7034 - val_accuracy: 0.7690
Epoch 5/20
26/26 [==============================] - 58s 2s/step - loss: 0.6779 - accuracy: 0.7735 - val_loss: 0.6802 - val_accuracy: 0.7683
Epoch 6/20
26/26 [==============================] - 58s 2s/step - loss: 0.6248 - accuracy: 0.7847 - val_loss: 0.6889 - val_accuracy: 0.7690


In [30]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 12s 61ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.34      0.93      0.50       683
           2       0.14      0.72      0.23       225
           3       0.13      0.88      0.23       273
           4       0.10      0.90      0.19       641
           5       0.78      0.98      0.87      4375

   micro avg       0.37      0.95      0.53      6197
   macro avg       0.25      0.74      0.34      6197
weighted avg       0.61      0.95      0.71      6197
 samples avg       0.41      0.95      0.57      6197



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Сравнение результатов

In [31]:
comparing_list

[['CNN+pooling', 0.8865749835968018, 0.748271107673645],
 ['RNN', 0.8310151100158691, 0.7558782696723938],
 ['CNN+RNN', 0.8363175392150879, 0.7475795149803162],
 ['CNN+pooling+RNN', 0.833013117313385, 0.770401120185852],
 ['RNN+CNN+CNN', 0.8183355331420898, 0.7475795149803162],
 ['RNN+CNN+pooling+RNN', 0.7786059975624084, 0.7634854912757874],
 ['(CNN+RNN)x3+polling', 0.7846768498420715, 0.7690179944038391]]

In [32]:
pd.DataFrame(comparing_list, columns=['net_name', 'accuracy', 'val_accuracy'])

,net_name,accuracy,val_accuracy
0,CNN+pooling,0.886575,0.748271
1,RNN,0.831015,0.755878
2,CNN+RNN,0.836318,0.747580
3,CNN+pooling+RNN,0.833013,0.770401
4,RNN+CNN+CNN,0.818336,0.747580
5,RNN+CNN+pooling+RNN,0.778606,0.763485
6,(CNN+RNN)x3+polling,0.784677,0.769018


как ни смешно, но обычная RNN уделала остальные архитектуры по метрике качества. Она же относительно других архитектур неплохо предсказывает и классы кроме 5 это (1,2,3,4)